# LSTM Model Local Testing

This notebook is for local testing of the LSTM model before Kaggle submission.

## Purpose
- Test model loading and inference
- Verify prediction functionality
- Check performance with local test data
- Validate the submission pipeline locally


In [2]:
import os
import polars as pl
import tensorflow as tf
import sys

# Add the kaggle_evaluation directory to the path
sys.path.append('../../data/')

# Import CMI evaluation system
import kaggle_evaluation.cmi_inference_server

# Import our inference module
from model_inference import predict_gesture

print("Libraries imported successfully")
print(f"TensorFlow version: {tf.__version__}")


Libraries imported successfully
TensorFlow version: 2.19.0


In [3]:
def predict(sequence: pl.DataFrame, demographics: pl.DataFrame) -> str:
    """
    Main prediction function for local testing
    
    Args:
        sequence: Polars DataFrame containing sensor data sequence
        demographics: Polars DataFrame containing participant demographics
    
    Returns:
        str: Predicted gesture name
    """
    # Convert Polars DataFrame to Pandas DataFrame
    sequence_df = sequence.to_pandas()
    
    # Predict gesture using our trained LSTM model
    gesture = predict_gesture(sequence_df)
    
    return gesture


print("Local prediction function defined")


Local prediction function defined


In [4]:
# Initialize the inference server for local testing
inference_server = kaggle_evaluation.cmi_inference_server.CMIInferenceServer(predict)

# Run local testing
print("Starting local testing...")
inference_server.run_local_gateway(
    data_paths=(
        '../../data/test.csv',
        '../../data/test_demographics.csv',
    )
)


Starting local testing...


I0000 00:00:1751798779.551584  123502 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5660 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050, pci bus id: 0000:01:00.0, compute capability: 8.6


Pre-trained LSTM model loaded successfully
Model input shape: (None, 64, 332)
Model output shape: (None, 18)
Number of classes: 18
Window size: 64


I0000 00:00:1751798781.497863  123533 cuda_dnn.cc:529] Loaded cuDNN version 90501


In [7]:
import pandas as pd
df = pd.read_parquet('submission.parquet')
print('Shape:', df.shape)
print('Columns:', df.columns.tolist())
print('First 10 rows:')
print(df.head(10))
print('Value counts:')
print(df['gesture'].value_counts())

Shape: (2, 2)
Columns: ['sequence_id', 'gesture']
First 10 rows:
  sequence_id              gesture
0  SEQ_000001    Neck - pinch skin
1  SEQ_000011  Eyelash - pull hair
Value counts:
gesture
Neck - pinch skin      1
Eyelash - pull hair    1
Name: count, dtype: int64
